# Linked Lists

*Dynamic data structures with pointer-based connections*


## 🎯 Learning Objectives

- Understand why linked lists exist and their trade-offs compared to arrays
- Implement singly linked lists with insertion, deletion, search, and traversal operations
- Build doubly linked and circular linked list variants
- Apply key linked list patterns: dummy head, fast/slow pointers, and in-place reversal
- Solve classic linked list problems including cycle detection, merging sorted lists, and list partitioning

---
## 1. Why Linked Lists?


In the previous week, we learned about arrays and their O(1) random access. Arrays are powerful, but they have a significant limitation: inserting or deleting elements in the middle requires shifting all subsequent elements—an O(n) operation. **Linked lists** solve this problem by trading random access for efficient insertions and deletions.

> 📖 **Definition:** A **linked list** is a linear data structure where elements are stored in nodes, and each node contains a reference (pointer) to the next node in the sequence. Unlike arrays, linked list elements are not stored in contiguous memory locations.

### The Problem with Arrays

> 💡 **Array Insertion Problem**
>
> Consider inserting an element at the beginning of an array with 1 million elements:
```

Before: [A, B, C, D, E, ...]  (1 million elements)
Insert X at position 0

Step 1: Shift E right    [A, B, C, D, _, E, ...]
Step 2: Shift D right    [A, B, C, _, D, E, ...]
Step 3: Shift C right    [A, B, _, C, D, E, ...]
...
Step 999,999: Shift A    [_, A, B, C, D, E, ...]
Step 1,000,000: Insert X [X, A, B, C, D, E, ...]

→ 1 million operations just to insert one element!
                
```

With a linked list, inserting at the beginning takes only O(1) time—we simply create a new node and update one pointer. No shifting required.

### Array vs Linked List: Trade-offs

| Operation | Array | Linked List |
| --- | --- | --- |
| Access by index | O(1) | O(n) |
| Insert at beginning | O(n) | O(1) |
| Insert at end | O(1)* | O(n) or O(1)** |
| Insert in middle | O(n) | O(1)*** |
| Delete at beginning | O(n) | O(1) |
| Search | O(n) | O(n) |
| Memory | Compact | Extra pointer overhead |

** Amortized for dynamic arrays. ** O(1) with tail pointer. *** Once you have a reference to the position.*

**Listing 4.1 — Demonstrating the Trade-off**

In [ ]:
import time

# Simulate the difference: insert at beginning
n = 10000

# Array (Python list): Insert at beginning is O(n)
arr = list(range(n))
start = time.perf_counter()
for i in range(100):
    arr.insert(0, -1)  # Insert at beginning
array_time = time.perf_counter() - start

# Linked list simulation: Insert at beginning is O(1)
# We'll use a collections.deque which has O(1) left operations
from collections import deque
linked = deque(range(n))
start = time.perf_counter()
for i in range(100):
    linked.appendleft(-1)  # Insert at beginning
linked_time = time.perf_counter() - start

print(f"Insert 100 elements at beginning (n={n}):")
print(f"  Array (list.insert): {array_time*1000:.2f} ms")
print(f"  Linked (deque):      {linked_time*1000:.2f} ms")
speedup = array_time/linked_time if linked_time > 0 else float('inf')
print(f"  Speedup: {speedup:.1f}x")

***Figure 4.1:** Inserting at the beginning shows the linked list advantage. The speedup grows with n.*

**Listing 4.2 — When to Use Each**

In [ ]:
# Decision guide: Array vs Linked List

def demonstrate_use_cases():
    """Show when each data structure excels."""
    
    print("USE ARRAY (list) WHEN:")
    print("  - Frequent random access by index")
    print("  - Known or fixed size")
    print("  - Memory efficiency matters")
    print("  - Cache locality important (iteration)")
    
    print("\nUSE LINKED LIST WHEN:")
    print("  - Frequent insertions/deletions at front")
    print("  - Unknown size that changes often")
    print("  - Memory allocation in small chunks preferred")
    print("  - No random access needed")
    
    print("\n" + "="*50)
    print("PRACTICAL EXAMPLES:")
    print("="*50)
    
    print("\nArray: Store student grades, access by ID")
    grades = [85, 90, 78, 92, 88]  # O(1) access
    print(f"  grades[2] = {grades[2]}")
    
    print("\nLinked List: Undo history (add/remove at ends)")
    from collections import deque
    history = deque()
    history.append("action1")  # O(1)
    history.append("action2")  # O(1)
    print(f"  Undo: {history.pop()}")  # O(1) from end

demonstrate_use_cases()

***Figure 4.2:** Choose arrays for random access; linked lists for frequent head/tail operations.*

---
## 2. Nodes: The Building Block


Every linked list is built from **nodes**. A node is a simple container that holds two things: the data value and a reference to the next node.

> 📖 **Definition:** A **node** is the fundamental unit of a linked list. It contains: (1) a **data** field storing the actual value, and (2) a **next** field storing a reference (pointer) to the next node in the list, or `None` if it's the last node.

**Listing 4.2 — The Node Class**

In [ ]:
# The Node class - building block of linked lists
class Node:
    """A node in a singly linked list."""
    
    def __init__(self, data):
        self.data = data    # The value stored in this node
        self.next = None    # Reference to the next node (initially None)
    
    def __repr__(self):
        return f"Node({self.data})"

# Create some nodes
node1 = Node(10)
node2 = Node(20)
node3 = Node(30)

print(f"node1: {node1}, next = {node1.next}")
print(f"node2: {node2}, next = {node2.next}")
print(f"node3: {node3}, next = {node3.next}")

# Link them together: 10 -> 20 -> 30
node1.next = node2
node2.next = node3

print("\nAfter linking:")
print(f"node1.next = {node1.next}")
print(f"node2.next = {node2.next}")
print(f"node3.next = {node3.next}")

# Traverse the chain
print("\nTraversal:")
current = node1
while current:
    print(f"  {current.data}", end=" -> " if current.next else "\n")
    current = current.next

***Figure 4.2:** Nodes are linked by setting each node's `next` to reference the following node.*

### Understanding References

In Python, variables hold *references* to objects, not the objects themselves. When we write `node1.next = node2`, we're storing a reference to `node2` inside `node1`. Both `node1.next` and `node2` point to the same object in memory.

**Listing 4.3 — References and Identity**

In [ ]:
# Understanding references in linked lists
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

# Create nodes
a = Node("A")
b = Node("B")

# Link them
a.next = b

# a.next and b refer to the SAME object
print(f"a.next is b: {a.next is b}")
print(f"id(a.next): {id(a.next)}")
print(f"id(b):      {id(b)}")

# Modifying through one reference affects the other
a.next.data = "B-modified"
print(f"\nAfter a.next.data = 'B-modified':")
print(f"  b.data = '{b.data}'")  # Also changed!

# This is why linked lists work - nodes "know" each other through references

***Figure 4.3:** References allow nodes to form chains. Changes through one reference affect the same object.*

### Visualizing a Linked List

> 💡 **Linked List Visualization**
>
> ```

head
  │
  ▼
┌─────────┐    ┌─────────┐    ┌─────────┐    ┌─────────┐
│ data: 1 │    │ data: 2 │    │ data: 3 │    │ data: 4 │
│ next: ──┼───►│ next: ──┼───►│ next: ──┼───►│ next: ──┼───► None
└─────────┘    └─────────┘    └─────────┘    └─────────┘
   Node 0        Node 1         Node 2         Node 3

Memory addresses are NOT contiguous:
  Node 0 at 0x7f001234
  Node 1 at 0x7f009876  (could be anywhere!)
  Node 2 at 0x7f002468
  Node 3 at 0x7f008642
                
```

---
## 3. Singly Linked Lists


A **singly linked list** is the simplest form of linked list, where each node has only one link—to the next node. The list maintains a reference to the first node, called the **head**.

**Listing 4.4 — Basic LinkedList Class**

In [ ]:
# Complete Singly Linked List implementation
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    """A singly linked list."""
    
    def __init__(self):
        self.head = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self.head is None
    
    def __str__(self):
        """String representation: [1 -> 2 -> 3]"""
        if self.is_empty():
            return "[]"
        
        result = []
        current = self.head
        while current:
            result.append(str(current.data))
            current = current.next
        return "[" + " -> ".join(result) + "]"
    
    def __iter__(self):
        """Allow iteration: for item in linked_list"""
        current = self.head
        while current:
            yield current.data
            current = current.next

# Test
ll = LinkedList()
print(f"Empty list: {ll}")
print(f"Length: {len(ll)}")
print(f"Is empty: {ll.is_empty()}")

***Figure 4.4:** The LinkedList class maintains a head pointer and size counter.*

**Listing 4.5 — LinkedList with Tail Pointer**

In [ ]:
# LinkedList with tail pointer for O(1) append
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedListWithTail:
    """Singly linked list with both head and tail pointers."""
    
    def __init__(self):
        self.head = None
        self.tail = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def __str__(self):
        if not self.head:
            return "[]"
        parts = []
        curr = self.head
        while curr:
            parts.append(str(curr.data))
            curr = curr.next
        return "[" + " -> ".join(parts) + "]"
    
    def append(self, data):
        """O(1) append with tail pointer!"""
        new_node = Node(data)
        self._size += 1
        
        if not self.head:
            self.head = self.tail = new_node
            return
        
        self.tail.next = new_node
        self.tail = new_node
    
    def prepend(self, data):
        """O(1) prepend."""
        new_node = Node(data)
        self._size += 1
        
        if not self.head:
            self.head = self.tail = new_node
            return
        
        new_node.next = self.head
        self.head = new_node

# Test: append is now O(1)!
ll = LinkedListWithTail()
for i in range(5):
    ll.append(i)

print(f"List: {ll}")
print(f"Head: {ll.head.data}, Tail: {ll.tail.data}")

***Figure 4.5:** A tail pointer makes append O(1) instead of O(n).*

### Adding to the List

**Listing 4.5 — Insertion Methods**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def __str__(self):
        if not self.head:
            return "[]"
        parts = []
        curr = self.head
        while curr:
            parts.append(str(curr.data))
            curr = curr.next
        return "[" + " -> ".join(parts) + "]"
    
    def prepend(self, data):
        """Insert at beginning - O(1)."""
        new_node = Node(data)
        new_node.next = self.head  # Point to old head
        self.head = new_node       # Update head
        self._size += 1
    
    def append(self, data):
        """Insert at end - O(n) without tail pointer."""
        new_node = Node(data)
        self._size += 1
        
        if not self.head:
            self.head = new_node
            return
        
        # Traverse to find last node
        current = self.head
        while current.next:
            current = current.next
        current.next = new_node
    
    def insert_at(self, index, data):
        """Insert at specific index - O(n)."""
        if index < 0 or index > self._size:
            raise IndexError("Index out of bounds")
        
        if index == 0:
            self.prepend(data)
            return
        
        new_node = Node(data)
        current = self.head
        for _ in range(index - 1):  # Stop at node before target
            current = current.next
        
        new_node.next = current.next
        current.next = new_node
        self._size += 1

# Test insertions
ll = LinkedList()
print("Building list:")

ll.append(1)
print(f"append(1): {ll}")

ll.append(3)
print(f"append(3): {ll}")

ll.prepend(0)
print(f"prepend(0): {ll}")

ll.insert_at(2, 2)
print(f"insert_at(2, 2): {ll}")

ll.append(4)
print(f"append(4): {ll}")

print(f"\nFinal list: {ll}")
print(f"Length: {len(ll)}")

***Figure 4.5:** Three insertion methods: prepend (O(1)), append (O(n)), and insert_at (O(n)).*

---
## 4. Basic Operations


### Traversal

**Listing 4.6 — Traversal Patterns**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

# Build a list manually for demonstration
head = Node(10)
head.next = Node(20)
head.next.next = Node(30)
head.next.next.next = Node(40)

# Pattern 1: Basic traversal
print("Basic traversal:")
current = head
while current:
    print(f"  {current.data}")
    current = current.next

# Pattern 2: Count nodes
count = 0
current = head
while current:
    count += 1
    current = current.next
print(f"\nNode count: {count}")

# Pattern 3: Find a value
def find(head, target):
    """Return index of target, or -1 if not found."""
    current = head
    index = 0
    while current:
        if current.data == target:
            return index
        current = current.next
        index += 1
    return -1

print(f"\nFind 30: index {find(head, 30)}")
print(f"Find 99: index {find(head, 99)}")

# Pattern 4: Get node at index
def get_at(head, index):
    """Return data at index, or None if out of bounds."""
    current = head
    for i in range(index):
        if current is None:
            return None
        current = current.next
    return current.data if current else None

print(f"\nGet index 2: {get_at(head, 2)}")
print(f"Get index 10: {get_at(head, 10)}")

***Figure 4.6:** Traversal patterns: basic loop, counting, searching, and indexed access.*

### Search Operations

**Listing 4.7 — Search Operations**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def find_all(head, target):
    """Find all indices where target appears."""
    indices = []
    current = head
    index = 0
    while current:
        if current.data == target:
            indices.append(index)
        current = current.next
        index += 1
    return indices

def find_min_max(head):
    """Find minimum and maximum values."""
    if not head:
        return None, None
    
    min_val = max_val = head.data
    current = head.next
    
    while current:
        if current.data < min_val:
            min_val = current.data
        if current.data > max_val:
            max_val = current.data
        current = current.next
    
    return min_val, max_val

def count_occurrences(head, target):
    """Count how many times target appears."""
    count = 0
    current = head
    while current:
        if current.data == target:
            count += 1
        current = current.next
    return count

# Test
head = build_list([3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5])
print(f"List: 3 -> 1 -> 4 -> 1 -> 5 -> 9 -> 2 -> 6 -> 5 -> 3 -> 5")
print(f"find_all(5): indices {find_all(head, 5)}")
print(f"find_min_max(): {find_min_max(head)}")
print(f"count_occurrences(5): {count_occurrences(head, 5)}")

***Figure 4.7:** Search operations for finding values, min/max, and counting occurrences.*

### Deletion

**Listing 4.7 — Deletion Operations**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None
        self._size = 0
    
    def __str__(self):
        if not self.head:
            return "[]"
        parts = []
        curr = self.head
        while curr:
            parts.append(str(curr.data))
            curr = curr.next
        return "[" + " -> ".join(parts) + "]"
    
    def append(self, data):
        new_node = Node(data)
        self._size += 1
        if not self.head:
            self.head = new_node
            return
        curr = self.head
        while curr.next:
            curr = curr.next
        curr.next = new_node
    
    def delete_first(self):
        """Delete first node - O(1)."""
        if not self.head:
            raise IndexError("Delete from empty list")
        data = self.head.data
        self.head = self.head.next
        self._size -= 1
        return data
    
    def delete_last(self):
        """Delete last node - O(n)."""
        if not self.head:
            raise IndexError("Delete from empty list")
        
        if not self.head.next:  # Only one node
            data = self.head.data
            self.head = None
            self._size -= 1
            return data
        
        # Find second-to-last node
        current = self.head
        while current.next.next:
            current = current.next
        
        data = current.next.data
        current.next = None
        self._size -= 1
        return data
    
    def delete_at(self, index):
        """Delete at index - O(n)."""
        if index < 0 or index >= self._size:
            raise IndexError("Index out of bounds")
        
        if index == 0:
            return self.delete_first()
        
        current = self.head
        for _ in range(index - 1):
            current = current.next
        
        data = current.next.data
        current.next = current.next.next
        self._size -= 1
        return data
    
    def delete_value(self, value):
        """Delete first occurrence of value."""
        if not self.head:
            return False
        
        if self.head.data == value:
            self.head = self.head.next
            self._size -= 1
            return True
        
        current = self.head
        while current.next:
            if current.next.data == value:
                current.next = current.next.next
                self._size -= 1
                return True
            current = current.next
        return False

# Test deletions
ll = LinkedList()
for x in [1, 2, 3, 4, 5]:
    ll.append(x)

print(f"Original: {ll}")

val = ll.delete_first()
print(f"delete_first(): removed {val}, list: {ll}")

val = ll.delete_last()
print(f"delete_last(): removed {val}, list: {ll}")

val = ll.delete_at(1)
print(f"delete_at(1): removed {val}, list: {ll}")

ll.append(3)
ll.append(3)
print(f"\nAfter adding 3s: {ll}")

ll.delete_value(3)
print(f"delete_value(3): {ll}")

***Figure 4.7:** Deletion requires finding the node *before* the target to update its `next` pointer.*

### Key Insight: The Previous Node

For both insertion and deletion, you need access to the node *before* the target position. This is because you need to update its `next` pointer. This is a fundamental pattern in linked list operations.

> 💡 **Deleting a Middle Node**
>
> ```

Delete node B from: A -> B -> C -> D

Step 1: Find node BEFORE B (node A)
        ┌─────┐    ┌─────┐    ┌─────┐    ┌─────┐
        │  A  │───►│  B  │───►│  C  │───►│  D  │
        └─────┘    └─────┘    └─────┘    └─────┘
           ▲
        prev

Step 2: Set prev.next = prev.next.next (skip B)
        ┌─────┐    ┌─────┐    ┌─────┐    ┌─────┐
        │  A  │────┼──────────►│  C  │───►│  D  │
        └─────┘    └─────┘    └─────┘    └─────┘
                      ▲
              B is now orphaned (garbage collected)

Result: A -> C -> D
                
```

---
## 5. Complexity Analysis


### Time Complexity Summary

| Operation | Singly (no tail) | Singly (with tail) | Doubly |
| --- | --- | --- | --- |
| Access by index | O(n) | O(n) | O(n) |
| Insert at head | O(1) | O(1) | O(1) |
| Insert at tail | O(n) | O(1) | O(1) |
| Insert at position | O(n) | O(n) | O(n) |
| Delete head | O(1) | O(1) | O(1) |
| Delete tail | O(n) | O(n)* | O(1) |
| Delete at position | O(n) | O(n) | O(n) |
| Search | O(n) | O(n) | O(n) |

** Even with a tail pointer, singly linked list delete-at-tail is O(n) because we need to find the second-to-last node.*

### Space Complexity

**Listing 4.8 — Memory Overhead Comparison**

In [ ]:
import sys

# Python list memory
n = 1000
py_list = list(range(n))
list_size = sys.getsizeof(py_list)

# Estimate linked list memory
# Each node: object overhead + data ref + next ref
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

node = Node(0)
node_size = sys.getsizeof(node) + sys.getsizeof(node.__dict__)

print(f"Storing {n} integers:")
print(f"  Python list total: {list_size:,} bytes")
print(f"  Single node size:  ~{node_size} bytes")
print(f"  Linked list estimate: ~{node_size * n:,} bytes")
print(f"\nLinked list overhead: ~{(node_size * n) / list_size:.1f}x more memory")
print("\nLinked lists use more memory but enable O(1) insertions!")

***Figure 4.8:** Linked lists have significant memory overhead due to pointer storage. Use them when insertion efficiency matters more than memory.*

---
## 6. Doubly Linked Lists


A **doubly linked list** extends the singly linked list by adding a `prev` pointer to each node, allowing traversal in both directions.

> 📖 **Definition:** A **doubly linked list** is a linked list where each node has two pointers: `next` (pointing to the next node) and `prev` (pointing to the previous node). This enables O(1) deletion from the tail and bidirectional traversal.

**Listing 4.9 — Doubly Linked List Node**

In [ ]:
# Doubly Linked List Node
class DNode:
    """A node in a doubly linked list."""
    def __init__(self, data):
        self.data = data
        self.next = None
        self.prev = None
    
    def __repr__(self):
        return f"DNode({self.data})"

# Create and link nodes
a = DNode("A")
b = DNode("B")
c = DNode("C")

# Link forward
a.next = b
b.next = c

# Link backward
b.prev = a
c.prev = b

print("Forward traversal:")
current = a
while current:
    print(f"  {current.data}")
    current = current.next

print("\nBackward traversal:")
current = c
while current:
    print(f"  {current.data}")
    current = current.prev

# Show bidirectional links
print(f"\nb.prev.data = '{b.prev.data}'")  # A
print(f"b.next.data = '{b.next.data}'"  )  # C

***Figure 4.9:** Doubly linked nodes can traverse both forward and backward.*

**Listing 4.10 — Complete Doubly Linked List**

In [ ]:
class DNode:
    def __init__(self, data):
        self.data = data
        self.next = None
        self.prev = None

class DoublyLinkedList:
    """A doubly linked list with head and tail pointers."""
    
    def __init__(self):
        self.head = None
        self.tail = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def __str__(self):
        if not self.head:
            return "[]"
        parts = []
        curr = self.head
        while curr:
            parts.append(str(curr.data))
            curr = curr.next
        return "[" + " <-> ".join(parts) + "]"
    
    def append(self, data):
        """Add to end - O(1) with tail pointer!"""
        new_node = DNode(data)
        self._size += 1
        
        if not self.head:
            self.head = self.tail = new_node
            return
        
        new_node.prev = self.tail
        self.tail.next = new_node
        self.tail = new_node
    
    def prepend(self, data):
        """Add to beginning - O(1)."""
        new_node = DNode(data)
        self._size += 1
        
        if not self.head:
            self.head = self.tail = new_node
            return
        
        new_node.next = self.head
        self.head.prev = new_node
        self.head = new_node
    
    def delete_first(self):
        """Remove from beginning - O(1)."""
        if not self.head:
            raise IndexError("Empty list")
        
        data = self.head.data
        self.head = self.head.next
        
        if self.head:
            self.head.prev = None
        else:
            self.tail = None  # List is now empty
        
        self._size -= 1
        return data
    
    def delete_last(self):
        """Remove from end - O(1) with doubly linked!"""
        if not self.tail:
            raise IndexError("Empty list")
        
        data = self.tail.data
        self.tail = self.tail.prev
        
        if self.tail:
            self.tail.next = None
        else:
            self.head = None  # List is now empty
        
        self._size -= 1
        return data

# Test
dll = DoublyLinkedList()
for x in [1, 2, 3, 4, 5]:
    dll.append(x)

print(f"List: {dll}")
print(f"Head: {dll.head.data}, Tail: {dll.tail.data}")

print(f"\ndelete_last(): {dll.delete_last()}")
print(f"List: {dll}")

print(f"\ndelete_first(): {dll.delete_first()}")
print(f"List: {dll}")

***Figure 4.12:** Doubly linked list with tail pointer enables O(1) operations at both ends.*

**Listing 4.13 — Doubly Linked List Delete at Position**

In [ ]:
class DNode:
    def __init__(self, data):
        self.data = data
        self.next = None
        self.prev = None

class DoublyLinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
        self._size = 0
    
    def __str__(self):
        if not self.head:
            return "[]"
        parts = []
        curr = self.head
        while curr:
            parts.append(str(curr.data))
            curr = curr.next
        return "[" + " <-> ".join(parts) + "]"
    
    def append(self, data):
        new_node = DNode(data)
        self._size += 1
        if not self.head:
            self.head = self.tail = new_node
            return
        new_node.prev = self.tail
        self.tail.next = new_node
        self.tail = new_node
    
    def delete_node(self, node):
        """Delete a specific node in O(1) if you have reference!"""
        if node.prev:
            node.prev.next = node.next
        else:
            self.head = node.next
        
        if node.next:
            node.next.prev = node.prev
        else:
            self.tail = node.prev
        
        self._size -= 1
        return node.data
    
    def get_node_at(self, index):
        """Get node at index - can traverse from either end!"""
        if index < 0 or index >= self._size:
            return None
        
        # Optimize: start from closer end
        if index < self._size // 2:
            curr = self.head
            for _ in range(index):
                curr = curr.next
        else:
            curr = self.tail
            for _ in range(self._size - 1 - index):
                curr = curr.prev
        
        return curr

# Test
dll = DoublyLinkedList()
for x in [1, 2, 3, 4, 5]:
    dll.append(x)

print(f"List: {dll}")

# Get and delete middle node
middle = dll.get_node_at(2)
print(f"Middle node (index 2): {middle.data}")
dll.delete_node(middle)
print(f"After delete: {dll}")

***Figure 4.13:** Doubly linked list can delete any node in O(1) if you have a reference to it.*

---
## 7. Circular Linked Lists


In a **circular linked list**, the last node's `next` pointer points back to the first node, forming a circle. This is useful for round-robin scheduling, circular buffers, and games.

**Listing 4.11 — Circular Linked List**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class CircularLinkedList:
    """A circular singly linked list."""
    
    def __init__(self):
        self.head = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self.head is None
    
    def append(self, data):
        """Add to end (before head in circle)."""
        new_node = Node(data)
        self._size += 1
        
        if not self.head:
            self.head = new_node
            new_node.next = new_node  # Points to itself
            return
        
        # Find last node (the one pointing to head)
        current = self.head
        while current.next != self.head:
            current = current.next
        
        current.next = new_node
        new_node.next = self.head
    
    def display(self, max_iterations=None):
        """Display the circular list."""
        if not self.head:
            return "[]"
        
        parts = []
        current = self.head
        count = 0
        max_show = max_iterations or self._size
        
        while count < max_show:
            parts.append(str(current.data))
            current = current.next
            count += 1
            if current == self.head and max_iterations is None:
                break
        
        return "[" + " -> ".join(parts) + " -> (back to head)]"
    
    def rotate(self):
        """Move head to next node - O(1)."""
        if self.head:
            self.head = self.head.next

# Test
cll = CircularLinkedList()
for x in ["A", "B", "C", "D"]:
    cll.append(x)

print(f"Circular list: {cll.display()}")
print(f"Head: {cll.head.data}")

cll.rotate()
print(f"\nAfter rotate(): {cll.display()}")
print(f"Head: {cll.head.data}")

cll.rotate()
print(f"\nAfter rotate(): {cll.display()}")
print(f"Head: {cll.head.data}")

# Demonstrate circular nature
print("\nTraversing 10 times around the circle:")
current = cll.head
path = []
for i in range(10):
    path.append(current.data)
    current = current.next
print(" -> ".join(path))

***Figure 4.15:** Circular lists have no end—the last node points back to the head.*

**Listing 4.16 — Josephus Problem (Circular List)**

In [ ]:
# Classic Josephus Problem using circular list
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def josephus(n, k):
    """
    N people in circle, eliminate every k-th person.
    Return the survivor's position (1-indexed).
    """
    # Build circular list
    head = Node(1)
    current = head
    for i in range(2, n + 1):
        current.next = Node(i)
        current = current.next
    current.next = head  # Make circular
    
    # Eliminate until one remains
    prev = current  # Last node (before head)
    current = head
    
    while current.next != current:  # More than one node
        # Skip k-1 people
        for _ in range(k - 1):
            prev = current
            current = current.next
        
        # Eliminate current
        print(f"  Eliminated: {current.data}")
        prev.next = current.next
        current = prev.next
    
    return current.data

# Test: 7 people, eliminate every 3rd
print("Josephus(7, 3):")
survivor = josephus(7, 3)
print(f"  Survivor: Person {survivor}")

***Figure 4.16:** The Josephus problem is a classic application of circular linked lists.*

### Applications of Circular Lists

- **Round-robin scheduling:** CPU scheduling where each process gets equal time
- **Circular buffers:** Audio/video streaming buffers that wrap around
- **Game turns:** Cycling through players in a multiplayer game
- **Music playlists:** Repeat mode that loops back to start

---
## 8. Common Patterns & Techniques


### Pattern 1: Dummy Head Node

Using a dummy (sentinel) head node simplifies edge cases by ensuring there's always a node before the first real element.

**Listing 4.12 — Dummy Head Pattern**

In [ ]:
class Node:
    def __init__(self, data=None):
        self.data = data
        self.next = None

# WITHOUT dummy head - need special case for empty list
def insert_without_dummy(head, data):
    """Insert in sorted order (no dummy)."""
    new_node = Node(data)
    
    # Special case: empty list or insert at beginning
    if not head or data < head.data:
        new_node.next = head
        return new_node  # New head!
    
    # Find insertion point
    current = head
    while current.next and current.next.data < data:
        current = current.next
    
    new_node.next = current.next
    current.next = new_node
    return head

# WITH dummy head - no special cases needed!
def insert_with_dummy(dummy, data):
    """Insert in sorted order (with dummy)."""
    new_node = Node(data)
    
    # Always start from dummy - first real node is dummy.next
    current = dummy
    while current.next and current.next.data < data:
        current = current.next
    
    new_node.next = current.next
    current.next = new_node
    # No need to return new head - dummy is always the head

# Demonstrate
dummy = Node()  # Dummy head with no data
for x in [30, 10, 50, 20, 40]:
    insert_with_dummy(dummy, x)

# Print (skip dummy)
print("Sorted list with dummy head:")
current = dummy.next  # Skip dummy
while current:
    print(f"  {current.data}")
    current = current.next

***Figure 4.12:** A dummy head eliminates special cases for empty list and head insertion.*

### Pattern 2: Two-Pointer (Fast/Slow)

The fast and slow pointer technique is essential for many linked list problems.

**Listing 4.13 — Fast/Slow Pointers**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    """Helper to build a linked list from values."""
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def find_middle(head):
    """Find middle node using fast/slow pointers."""
    if not head:
        return None
    
    slow = fast = head
    while fast and fast.next:
        slow = slow.next          # Move 1 step
        fast = fast.next.next     # Move 2 steps
    
    return slow  # When fast reaches end, slow is at middle

def find_nth_from_end(head, n):
    """Find nth node from end using two pointers."""
    # Move fast pointer n nodes ahead
    fast = head
    for _ in range(n):
        if not fast:
            return None
        fast = fast.next
    
    # Move both until fast reaches end
    slow = head
    while fast:
        slow = slow.next
        fast = fast.next
    
    return slow

# Test find_middle
head = build_list([1, 2, 3, 4, 5])
print("List: 1 -> 2 -> 3 -> 4 -> 5")
middle = find_middle(head)
print(f"Middle: {middle.data}")

head = build_list([1, 2, 3, 4, 5, 6])
print("\nList: 1 -> 2 -> 3 -> 4 -> 5 -> 6")
middle = find_middle(head)
print(f"Middle: {middle.data}")

# Test find_nth_from_end
head = build_list([1, 2, 3, 4, 5])
print("\nList: 1 -> 2 -> 3 -> 4 -> 5")
for n in [1, 2, 3]:
    node = find_nth_from_end(head, n)
    print(f"  {n}th from end: {node.data}")

***Figure 4.13:** Fast/slow pointers find the middle in one pass. Gap technique finds nth from end.*

### Pattern 3: Runner Technique for Rearrangement

**Listing 4.14 — Reorder List**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts)

def reorder_list(head):
    """
    Reorder: L0 → L1 → ... → Ln-1 → Ln
    To:      L0 → Ln → L1 → Ln-1 → L2 → Ln-2 → ...
    """
    if not head or not head.next:
        return head
    
    # Step 1: Find middle
    slow, fast = head, head
    while fast.next and fast.next.next:
        slow = slow.next
        fast = fast.next.next
    
    # Step 2: Reverse second half
    prev, curr = None, slow.next
    slow.next = None  # Cut the list
    while curr:
        next_temp = curr.next
        curr.next = prev
        prev = curr
        curr = next_temp
    
    # Step 3: Merge two halves
    first, second = head, prev
    while second:
        tmp1, tmp2 = first.next, second.next
        first.next = second
        second.next = tmp1
        first, second = tmp1, tmp2
    
    return head

# Test
head = build_list([1, 2, 3, 4, 5])
print(f"Original: {list_to_string(head)}")
reorder_list(head)
print(f"Reordered: {list_to_string(head)}")

***Figure 4.14:** Reordering combines finding middle, reversing, and merging.*

### Pattern 4: Reversing a Linked List

**Listing 4.14b — Reversing a Linked List**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts) if parts else "empty"

def reverse_iterative(head):
    """Reverse linked list iteratively - O(n) time, O(1) space."""
    prev = None
    current = head
    
    while current:
        next_temp = current.next  # Save next
        current.next = prev       # Reverse link
        prev = current            # Move prev forward
        current = next_temp       # Move current forward
    
    return prev  # New head

def reverse_recursive(head):
    """Reverse linked list recursively - O(n) time, O(n) space."""
    # Base case: empty or single node
    if not head or not head.next:
        return head
    
    # Reverse the rest
    new_head = reverse_recursive(head.next)
    
    # Put first element at end
    head.next.next = head
    head.next = None
    
    return new_head

# Test iterative
head = build_list([1, 2, 3, 4, 5])
print(f"Original:  {list_to_string(head)}")
head = reverse_iterative(head)
print(f"Reversed:  {list_to_string(head)}")

# Test recursive
head = build_list(["A", "B", "C", "D"])
print(f"\nOriginal:  {list_to_string(head)}")
head = reverse_recursive(head)
print(f"Reversed:  {list_to_string(head)}")

***Figure 4.14b:** Reversing a linked list is a fundamental operation. Iterative uses O(1) space; recursive uses O(n) stack space.*

---
## 9. Classic Problems


### Cycle Detection (Floyd's Algorithm)

**Listing 4.15 — Detecting Cycles**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def has_cycle(head):
    """
    Detect cycle using Floyd's algorithm.
    Fast pointer moves 2 steps, slow moves 1 step.
    If there's a cycle, they will eventually meet.
    """
    if not head:
        return False
    
    slow = fast = head
    
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        
        if slow == fast:  # They met - cycle exists!
            return True
    
    return False  # Fast reached end - no cycle

def find_cycle_start(head):
    """Find where the cycle begins."""
    if not head:
        return None
    
    # Phase 1: Detect cycle
    slow = fast = head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        if slow == fast:
            break
    else:
        return None  # No cycle
    
    # Phase 2: Find cycle start
    # Reset slow to head, move both at same speed
    slow = head
    while slow != fast:
        slow = slow.next
        fast = fast.next
    
    return slow  # Cycle start

# Create list with cycle: 1 -> 2 -> 3 -> 4 -> 5 -> 3 (back to 3)
head = Node(1)
head.next = Node(2)
head.next.next = Node(3)
head.next.next.next = Node(4)
head.next.next.next.next = Node(5)
head.next.next.next.next.next = head.next.next  # Cycle to node 3

print("List with cycle (1->2->3->4->5->3...):")
print(f"  has_cycle: {has_cycle(head)}")
cycle_start = find_cycle_start(head)
print(f"  cycle_start: {cycle_start.data}")

# Create list without cycle
head2 = Node(1)
head2.next = Node(2)
head2.next.next = Node(3)

print("\nList without cycle (1->2->3):")
print(f"  has_cycle: {has_cycle(head2)}")

***Figure 4.15:** Floyd's cycle detection uses O(1) space. If fast and slow meet, there's a cycle.*

### Merge Two Sorted Lists

**Listing 4.16 — Merging Sorted Lists**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts) if parts else "empty"

def merge_sorted(l1, l2):
    """Merge two sorted lists into one sorted list."""
    # Use dummy head to simplify edge cases
    dummy = Node(0)
    current = dummy
    
    while l1 and l2:
        if l1.data <= l2.data:
            current.next = l1
            l1 = l1.next
        else:
            current.next = l2
            l2 = l2.next
        current = current.next
    
    # Attach remaining nodes
    current.next = l1 or l2
    
    return dummy.next

# Test
list1 = build_list([1, 3, 5, 7])
list2 = build_list([2, 4, 6, 8])

print(f"List 1: {list_to_string(list1)}")
print(f"List 2: {list_to_string(list2)}")

merged = merge_sorted(list1, list2)
print(f"Merged: {list_to_string(merged)}")

***Figure 4.17:** Merging sorted lists is O(n+m) time and O(1) extra space (just reusing existing nodes).*

### Remove Nth Node from End

**Listing 4.18 — Remove Nth from End**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts)

def remove_nth_from_end(head, n):
    """Remove nth node from end (1-indexed)."""
    # Use dummy to handle edge case of removing head
    dummy = Node(0)
    dummy.next = head
    
    # Move fast n+1 steps ahead
    fast = slow = dummy
    for _ in range(n + 1):
        fast = fast.next
    
    # Move both until fast reaches end
    while fast:
        fast = fast.next
        slow = slow.next
    
    # Remove the node
    slow.next = slow.next.next
    
    return dummy.next

# Test
head = build_list([1, 2, 3, 4, 5])
print(f"Original: {list_to_string(head)}")

head = remove_nth_from_end(head, 2)
print(f"Remove 2nd from end: {list_to_string(head)}")

head = remove_nth_from_end(head, 1)
print(f"Remove 1st from end: {list_to_string(head)}")

***Figure 4.18:** Two-pointer technique with dummy head for safe removal from any position.*

### Swap Nodes in Pairs

**Listing 4.19 — Swap Nodes in Pairs**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts)

def swap_pairs(head):
    """Swap every two adjacent nodes."""
    dummy = Node(0)
    dummy.next = head
    prev = dummy
    
    while prev.next and prev.next.next:
        first = prev.next
        second = prev.next.next
        
        # Swap
        first.next = second.next
        second.next = first
        prev.next = second
        
        prev = first
    
    return dummy.next

# Test
head = build_list([1, 2, 3, 4, 5])
print(f"Original: {list_to_string(head)}")
head = swap_pairs(head)
print(f"Swapped pairs: {list_to_string(head)}")

***Figure 4.19:** Swapping adjacent pairs requires careful pointer manipulation.*

### Partition List

**Listing 4.20 — Partition List**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts)

def partition(head, x):
    """
    Partition list so all nodes < x come before nodes >= x.
    Maintains original relative order.
    """
    # Two dummy heads for two lists
    less_dummy = Node(0)
    greater_dummy = Node(0)
    less = less_dummy
    greater = greater_dummy
    
    current = head
    while current:
        if current.data < x:
            less.next = current
            less = less.next
        else:
            greater.next = current
            greater = greater.next
        current = current.next
    
    # Connect the two lists
    greater.next = None  # Important: terminate the list
    less.next = greater_dummy.next
    
    return less_dummy.next

# Test
head = build_list([1, 4, 3, 2, 5, 2])
print(f"Original: {list_to_string(head)}")
head = partition(head, 3)
print(f"Partitioned (x=3): {list_to_string(head)}")

***Figure 4.20:** Partition uses two separate lists for elements less than and greater than pivot.*

### Checking for Palindrome

**Listing 4.17 — Palindrome Check**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def reverse(head):
    prev = None
    while head:
        next_temp = head.next
        head.next = prev
        prev = head
        head = next_temp
    return prev

def is_palindrome(head):
    """
    Check if linked list is palindrome.
    Strategy: Find middle, reverse second half, compare.
    """
    if not head or not head.next:
        return True
    
    # Find middle using fast/slow
    slow = fast = head
    while fast.next and fast.next.next:
        slow = slow.next
        fast = fast.next.next
    
    # Reverse second half
    second_half = reverse(slow.next)
    
    # Compare first and second half
    first_half = head
    while second_half:
        if first_half.data != second_half.data:
            return False
        first_half = first_half.next
        second_half = second_half.next
    
    return True

# Test cases
test_cases = [
    [1, 2, 3, 2, 1],      # Palindrome (odd)
    [1, 2, 2, 1],          # Palindrome (even)
    [1, 2, 3, 4, 5],       # Not palindrome
    [1],                   # Single element
    [1, 1],                # Two same
    [1, 2],                # Two different
]

print("Palindrome checks:")
for values in test_cases:
    head = build_list(values)
    result = is_palindrome(head)
    print(f"  {values}: {result}")

***Figure 4.21:** Palindrome check uses fast/slow to find middle, then reverses and compares. O(n) time, O(1) space.*

### Copy List with Random Pointer

**Listing 4.22 — Deep Copy with Random Pointers**

In [ ]:
class RandomNode:
    def __init__(self, data):
        self.data = data
        self.next = None
        self.random = None

def copy_random_list(head):
    """
    Deep copy a list where each node has a random pointer.
    Uses O(n) space with hash map approach.
    """
    if not head:
        return None
    
    # Create mapping from original to copy
    old_to_new = {}
    
    # First pass: create all nodes
    current = head
    while current:
        old_to_new[current] = RandomNode(current.data)
        current = current.next
    
    # Second pass: set next and random pointers
    current = head
    while current:
        copy = old_to_new[current]
        copy.next = old_to_new.get(current.next)
        copy.random = old_to_new.get(current.random)
        current = current.next
    
    return old_to_new[head]

# Test
a = RandomNode(1)
b = RandomNode(2)
c = RandomNode(3)
a.next, b.next = b, c
a.random, b.random, c.random = c, a, b

# Copy
copy_head = copy_random_list(a)

# Verify copy
print("Original: 1 -> 2 -> 3")
print(f"  a.random = {a.random.data}, b.random = {b.random.data}")

print("\nCopy created successfully!")
print(f"  copy.data = {copy_head.data}")
print(f"  copy.random.data = {copy_head.random.data}")
print(f"  copy is a: {copy_head is a}")  # Should be False

***Figure 4.22:** Deep copy with random pointers uses a hash map to track original-to-copy mapping.*

### Sort a Linked List

**Listing 4.23 — Merge Sort for Linked List**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts)

def get_middle(head):
    """Get middle node (for splitting)."""
    slow, fast = head, head.next
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
    return slow

def merge(l1, l2):
    """Merge two sorted lists."""
    dummy = Node(0)
    curr = dummy
    while l1 and l2:
        if l1.data < l2.data:
            curr.next = l1
            l1 = l1.next
        else:
            curr.next = l2
            l2 = l2.next
        curr = curr.next
    curr.next = l1 or l2
    return dummy.next

def sort_list(head):
    """Sort linked list using merge sort. O(n log n) time."""
    if not head or not head.next:
        return head
    
    # Split list in half
    mid = get_middle(head)
    right = mid.next
    mid.next = None
    
    # Recursively sort both halves
    left = sort_list(head)
    right = sort_list(right)
    
    # Merge sorted halves
    return merge(left, right)

# Test
head = build_list([4, 2, 1, 3, 5])
print(f"Original: {list_to_string(head)}")
head = sort_list(head)
print(f"Sorted: {list_to_string(head)}")

***Figure 4.23:** Merge sort is ideal for linked lists: O(n log n) time, O(log n) stack space.*

### Rotate List

**Listing 4.24 — Rotate List by K**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    current = head
    for v in values[1:]:
        current.next = Node(v)
        current = current.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts)

def rotate_right(head, k):
    """Rotate list to the right by k places."""
    if not head or not head.next or k == 0:
        return head
    
    # Find length and last node
    length = 1
    tail = head
    while tail.next:
        tail = tail.next
        length += 1
    
    # Normalize k
    k = k % length
    if k == 0:
        return head
    
    # Find new tail (length - k - 1 steps from head)
    new_tail = head
    for _ in range(length - k - 1):
        new_tail = new_tail.next
    
    # Rotate
    new_head = new_tail.next
    new_tail.next = None
    tail.next = head
    
    return new_head

# Test
head = build_list([1, 2, 3, 4, 5])
print(f"Original: {list_to_string(head)}")

head = rotate_right(head, 2)
print(f"Rotate right 2: {list_to_string(head)}")

***Figure 4.24:** Rotation makes the list circular temporarily, then breaks at the right position.*

---
## 10. Common Pitfalls


### Pitfall 1: Losing the Head Reference

**Listing 4.18 — Losing References**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

# BAD: Modifying head directly
def bad_traverse(head):
    """Wrong - loses head reference!"""
    while head:
        print(head.data, end=" ")
        head = head.next  # BAD: modifying parameter
    # head is now None! Original reference is lost

# GOOD: Use a separate current pointer
def good_traverse(head):
    """Correct - preserves head reference."""
    current = head  # Use separate variable
    while current:
        print(current.data, end=" ")
        current = current.next
    # head still points to first node

# Demonstrate
head = Node(1)
head.next = Node(2)
head.next.next = Node(3)

print("Good traverse (head preserved):")
good_traverse(head)
print(f"\nhead after: {head.data}")

# Note: Python passes references by value, so bad_traverse
# doesn't actually modify the caller's head variable,
# but inside the function you lose track of the start.

***Figure 4.18:** Always use a separate `current` variable for traversal to preserve the head reference.*

### Pitfall 2: Forgetting to Update Links

**Listing 4.19 — Link Update Errors**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

# BAD: Order of operations matters!
def bad_insert_after(node, data):
    """Wrong order - loses rest of list!"""
    new_node = Node(data)
    node.next = new_node      # Now new_node.next is None
    new_node.next = node.next  # This is new_node itself! Wrong!

# GOOD: Save the next reference first
def good_insert_after(node, data):
    """Correct order - preserves list."""
    new_node = Node(data)
    new_node.next = node.next  # First: point to what's after
    node.next = new_node       # Then: insert new node

# Demonstrate
head = Node("A")
head.next = Node("C")

print(f"Before: A -> C")

# Insert B between A and C
good_insert_after(head, "B")

# Print result
current = head
parts = []
while current:
    parts.append(current.data)
    current = current.next
print(f"After good_insert_after: {' -> '.join(parts)}")

***Figure 4.19:** When inserting, set the new node's next pointer BEFORE updating the previous node's next.*

### Pitfall 3: Null Pointer Errors

**Listing 4.20 — Null Checks**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

# BAD: No null check before accessing .next
def bad_find_second(head):
    """Crashes on empty list!"""
    return head.next.data  # AttributeError if head is None!

# GOOD: Always check for None
def good_find_second(head):
    """Safe with proper null checks."""
    if not head:
        return None
    if not head.next:
        return None
    return head.next.data

# BAD: Not checking in loop condition
def bad_find_last(head):
    """Crashes when list is empty!"""
    while head.next:  # Crashes if head is None
        head = head.next
    return head.data

# GOOD: Check before loop
def good_find_last(head):
    """Safe implementation."""
    if not head:
        return None
    while head.next:
        head = head.next
    return head.data

# Test
print("Safe null handling:")
print(f"  good_find_second(None): {good_find_second(None)}")
print(f"  good_find_last(None): {good_find_last(None)}")

head = Node(1)
print(f"  good_find_second(single): {good_find_second(head)}")
print(f"  good_find_last(single): {good_find_last(head)}")

***Figure 4.20:** Always check if a node is None before accessing its attributes.*

- **Linked lists** trade O(1) random access for O(1) insertions/deletions at known positions
- **Nodes** contain data and a reference to the next node
- **Singly linked** lists traverse forward only; **doubly linked** go both ways
- **Tail pointer** enables O(1) append; **doubly linked** enables O(1) delete at tail
- **Fast/slow pointers** find middle, detect cycles, find nth from end
- **Dummy head** pattern simplifies edge cases
- **Always** check for None before accessing node attributes

---
# 📝 Exercises


### Exercise 1: Implement get(index)  (⭐ Easy)

Add a `get(index)` method to LinkedList that returns the data at the given index, or raises IndexError if out of bounds.

**Expected:** (Expected: get(0) returns head.data, get(2) returns third element's data)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Traverse with Counter:**

                        Start at head and keep a counter. Increment as you move through nodes.
- **Hint 2 - Return When Found:**

                        When counter equals index, return `current.data`
- **Hint 3 - Handle Bounds:**

                        If you reach end (current is None) before finding index, raise `IndexError`
</details>

In [ ]:
# ✏️ [EX1]
# Implement get(index) for LinkedList

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None
        self._size = 0
    
    def append(self, data):
        new_node = Node(data)
        self._size += 1
        if not self.head:
            self.head = new_node
            return
        curr = self.head
        while curr.next:
            curr = curr.next
        curr.next = new_node
    
    def get(self, index):
        # Your code here
        pass

# Test your implementation (uncomment)
# ll = LinkedList()
# for x in [10, 20, 30, 40, 50]:
#     ll.append(x)
# print(f"get(0): {ll.get(0)}")  # Expected: 10
# print(f"get(2): {ll.get(2)}")  # Expected: 30
# print(f"get(4): {ll.get(4)}")  # Expected: 50

### Exercise 2: Remove Duplicates from Sorted List  (⭐⭐ Medium)

Write `remove_duplicates(head)` that removes all duplicates from a sorted linked list, leaving only distinct values.

**Expected:** (Expected: [1, 1, 2, 3, 3, 3, 4] → [1, 2, 3, 4])

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Sorted Property:**

                        Since the list is sorted, duplicates are always adjacent.
- **Hint 2 - Compare Adjacent:**

                        Compare `current.data` with `current.next.data`
- **Hint 3 - Skip Duplicates:**

                        If same, skip by: `current.next = current.next.next`

                        If different, move forward: `current = current.next`
</details>

In [ ]:
# ✏️ [EX2]
# Remove Duplicates from Sorted Linked List

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    curr = head
    for v in values[1:]:
        curr.next = Node(v)
        curr = curr.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts)

def remove_duplicates(head):
    # Your code here
    pass

# Test your implementation (uncomment)
# head = build_list([1, 1, 2, 3, 3, 3, 4])
# print(f"Before: {list_to_string(head)}")
# remove_duplicates(head)
# print(f"After: {list_to_string(head)}")

### Exercise 3: Add Two Numbers (as Linked Lists)  (⭐⭐ Medium)

Given two linked lists representing numbers in reverse order (each node contains a single digit), return a new linked list representing their sum.

**Expected:** (Expected: [2,4,3] + [5,6,4] = [7,0,8] representing 342 + 465 = 807)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Process Together:**

                        Traverse both lists simultaneously, adding corresponding digits.
- **Hint 2 - Track Carry:**

`carry = sum // 10`, `digit = sum % 10`
- **Hint 3 - Don't Forget Final Carry:**

                        After both lists end, if carry > 0, add one more node.
</details>

In [ ]:
# ✏️ [EX3]
# Add Two Numbers as Linked Lists

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    curr = head
    for v in values[1:]:
        curr.next = Node(v)
        curr = curr.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts)

def add_two_numbers(l1, l2):
    # Your code here
    pass

# Test your implementation (uncomment)
# l1 = build_list([2, 4, 3])  # Represents 342
# l2 = build_list([5, 6, 4])  # Represents 465
# result = add_two_numbers(l1, l2)
# print(f"{list_to_string(l1)}")
# print(f"{list_to_string(l2)}")
# print(f"Sum: {list_to_string(result)}")  # Should be 7 -> 0 -> 8

### Exercise 4: Intersection of Two Lists  (⭐⭐⭐ Hard)

Write `get_intersection(headA, headB)` that returns the node where two linked lists intersect, or None if they don't.

**Expected:** (Expected: Return the shared node, not just equal data)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Two-Pointer Technique:**

                        Use two pointers, one for each list. When one reaches end, redirect to the other list's head.
- **Hint 2 - Equal Path Lengths:**

                        Both pointers will travel same total distance (lenA + lenB), meeting at intersection.
- **Hint 3 - Alternative:**

                        Get lengths, advance longer list by difference, then traverse together.
</details>

In [ ]:
# ✏️ [EX4]
# Intersection of Two Linked Lists

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def get_intersection(headA, headB):
    # Your code here
    pass

# Test your implementation (uncomment)
# Create two lists that intersect:
# A: 1 -> 2 -> 3 \
#                 -> 6 -> 7 -> None
# B:      4 -> 5 /

# shared = Node(6)
# shared.next = Node(7)
# headA = Node(1)
# headA.next = Node(2)
# headA.next.next = Node(3)
# headA.next.next.next = shared
# headB = Node(4)
# headB.next = Node(5)
# headB.next.next = shared
# intersection = get_intersection(headA, headB)
# print(f"Intersection at: {intersection.data if intersection else None}")

### Exercise 5: Reverse Nodes in k-Groups  (⭐⭐⭐ Hard)

Write `reverse_k_group(head, k)` that reverses nodes in groups of k. If remaining nodes are less than k, leave them as is.

**Expected:** (Expected: [1,2,3,4,5] with k=2 → [2,1,4,3,5])

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Check Length First:**

                        Count if there are at least k nodes remaining. If not, return head as is.
- **Hint 2 - Reverse k Nodes:**

                        Use standard linked list reversal for exactly k nodes.
- **Hint 3 - Recursive Approach:**

                        After reversing k nodes, recursively process the rest.

                        Connect reversed group's tail to recursion result.
</details>

In [ ]:
# ✏️ [EX5]
# Reverse Nodes in k-Groups

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def build_list(values):
    if not values:
        return None
    head = Node(values[0])
    curr = head
    for v in values[1:]:
        curr.next = Node(v)
        curr = curr.next
    return head

def list_to_string(head):
    parts = []
    while head:
        parts.append(str(head.data))
        head = head.next
    return " -> ".join(parts)

def reverse_k_group(head, k):
    # Your code here
    pass

# Test your implementation (uncomment)
# head = build_list([1, 2, 3, 4, 5])
# print(f"Original: {list_to_string(head)}")
# result = reverse_k_group(head, 2)
# print(f"k=2: {list_to_string(result)}")  # 2 -> 1 -> 4 -> 3 -> 5

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,10}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-10 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request

WEEK = "Week_04"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ── Check info was filled in ──
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Run the cell above first to set your info!")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Run the cell above first — some fields are empty.")

# ── Extract exercise answers from IPython history ──
_answers = {}
try:
    _ipy = get_ipython()
    _hist = _ipy.history_manager.get_range(output=False)
    for _sess, _line, _src in _hist:
        _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
        if _m:
            _ex_id = "ex" + _m.group(1)
            _lines = _src.split("\n")
            _clean = "\n".join(_lines[1:]).strip()
            _answers[_ex_id] = {
                "code": _clean,
                "modified": len(_clean) > 5
            }
except Exception:
    pass

# ── Fallback: also check In[] from current session ──
if not _answers:
    try:
        for _src in In:
            if not _src:
                continue
            _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
            if _m:
                _ex_id = "ex" + _m.group(1)
                _lines = _src.split("\n")
                _clean = "\n".join(_lines[1:]).strip()
                _answers[_ex_id] = {
                    "code": _clean,
                    "modified": len(_clean) > 5
                }
    except NameError:
        pass

# ── Fallback: try reading notebook file (VS Code) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir(".") if f.endswith(".ipynb") and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), "r", encoding="utf-8") as _f:
            _nb = json.load(_f)
        for _cell in _nb["cells"]:
            if _cell["cell_type"] != "code":
                continue
            _src = "".join(_cell["source"]) if isinstance(_cell["source"], list) else _cell["source"]
            _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
            if _m:
                _ex_id = "ex" + _m.group(1)
                _lines = _src.split("\n")
                _clean = "\n".join(_lines[1:]).strip()
                _answers[_ex_id] = {
                    "code": _clean,
                    "modified": len(_clean) > 5
                }

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys()))}")

if not _answers:
    print("\n⚠️  No exercise answers found!")
    print("Make sure you RAN all exercise cells before submitting.")
    raise SystemExit()

# ── Send ──
_data = json.dumps({
    "week": WEEK,
    "studentId": _sid,
    "studentName": _sname,
    "studentEmail": _semail,
    "classCode": _scode,
    "source": "dsa-notebook",
    "timeOnPage": 0,
    "answers": _answers
}).encode("utf-8")

print("📡 Submitting...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get("success"):
        print(f"\n✅ {_result['message']}")
        print("📧 Check your email for confirmation.")
    else:
        print(f"\n❌ {_result.get('message', 'Submission failed')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  Request sent — check your email for confirmation.")
    print(f"(If no email arrives, try again or contact your instructor)")
